[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/buildLittleWorlds/ml-math-with-densworld/blob/main/modules/02-linear-algebra/notebooks/02-vector-norms.ipynb)

# Lesson 2: Vector Norms — Measuring Distance in Different Ways

*"The shortest path between two points depends on how you're allowed to travel. In the Dens, where tunnels twist and branch, the straight line is a fantasy. The Pickbox Man measures distance in footsteps along corridors. Vagabu Olt measures it as the crow flies—through solid rock, if necessary."*  
— The Pickbox Man, tunnel surveyor

---

## The Core Problem

In Lesson 1, we measured distance between creatures using the **Euclidean distance**—the straight-line path through feature space. But this isn't the only way to measure "how different" two things are.

Consider two mapmakers surveying the Dens:
- **Vagabu Olt** thinks in straight lines (Euclidean, L2)
- **The Pickbox Man** navigates through tunnels (Manhattan, L1)

Their different perspectives lead to **different distance measurements**—and in machine learning, the choice of distance metric has real consequences for classification, clustering, and regularization.

---

## Learning Objectives

By the end of this lesson, you will:
1. Calculate L1 (Manhattan) and L2 (Euclidean) norms and distances
2. Understand when each norm is more appropriate
3. See how norm choice affects nearest-neighbor classification
4. Connect norms to regularization in machine learning (L1 → Lasso, L2 → Ridge)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist

# Set random seed for reproducibility
np.random.seed(42)

# Nice plotting defaults
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

# Colab-ready data loading
BASE_URL = "https://raw.githubusercontent.com/buildLittleWorlds/ml-math-with-densworld/main/data/"

# Load our datasets
creature_vectors = pd.read_csv(BASE_URL + "creature_vectors.csv")
creature_similarity = pd.read_csv(BASE_URL + "creature_similarity.csv")
dens_boundary = pd.read_csv(BASE_URL + "dens_boundary_observations.csv")

print(f"Loaded {len(creature_vectors)} creatures with behavioral/habitat vectors")
print(f"Loaded {len(creature_similarity)} pairwise similarity calculations")
print(f"Loaded {len(dens_boundary)} boundary observations")

## Part 1: Two Ways to Measure — L1 vs L2

### The Euclidean Norm (L2)

The **L2 norm** measures the straight-line distance—"as the crow flies":

$$\|\mathbf{v}\|_2 = \sqrt{\sum_{i=1}^{n} v_i^2}$$

For the distance between two vectors:
$$d_{L2}(\mathbf{a}, \mathbf{b}) = \sqrt{\sum_{i=1}^{n} (a_i - b_i)^2}$$

### The Manhattan Norm (L1)

The **L1 norm** measures distance along axes—like walking a city grid:

$$\|\mathbf{v}\|_1 = \sum_{i=1}^{n} |v_i|$$

For the distance between two vectors:
$$d_{L1}(\mathbf{a}, \mathbf{b}) = \sum_{i=1}^{n} |a_i - b_i|$$

In [ ]:
# Visual comparison of L1 vs L2 distance
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Define two points
point_a = np.array([1, 1])
point_b = np.array([4, 5])

# L2 (Euclidean) - straight line
ax = axes[0]
ax.scatter(*point_a, s=150, c='steelblue', zorder=5, label='Point A (1,1)')
ax.scatter(*point_b, s=150, c='coral', zorder=5, label='Point B (4,5)')
ax.plot([point_a[0], point_b[0]], [point_a[1], point_b[1]], 'g-', linewidth=3, 
        label=f'L2 distance = {np.linalg.norm(point_b - point_a):.2f}')
ax.set_xlim(0, 6)
ax.set_ylim(0, 6)
ax.set_xlabel('x', fontsize=12)
ax.set_ylabel('y', fontsize=12)
ax.set_title('L2 (Euclidean): "As the Crow Flies"\nVagabu Olt\'s View', fontsize=13)
ax.legend(fontsize=10)
ax.set_aspect('equal')
ax.grid(True, alpha=0.3)

# L1 (Manhattan) - along grid
ax = axes[1]
ax.scatter(*point_a, s=150, c='steelblue', zorder=5, label='Point A (1,1)')
ax.scatter(*point_b, s=150, c='coral', zorder=5, label='Point B (4,5)')
# Draw the L1 path (horizontal then vertical)
ax.plot([point_a[0], point_b[0]], [point_a[1], point_a[1]], 'purple', linewidth=3)
ax.plot([point_b[0], point_b[0]], [point_a[1], point_b[1]], 'purple', linewidth=3,
        label=f'L1 distance = {np.sum(np.abs(point_b - point_a)):.2f}')
ax.set_xlim(0, 6)
ax.set_ylim(0, 6)
ax.set_xlabel('x', fontsize=12)
ax.set_ylabel('y', fontsize=12)
ax.set_title('L1 (Manhattan): "Through the Tunnels"\nThe Pickbox Man\'s View', fontsize=13)
ax.legend(fontsize=10)
ax.set_aspect('equal')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Calculate both distances
l2_dist = np.linalg.norm(point_b - point_a)
l1_dist = np.sum(np.abs(point_b - point_a))
print(f"L2 (Euclidean) distance: {l2_dist:.4f}")
print(f"L1 (Manhattan) distance: {l1_dist:.4f}")
print(f"\nL1 is always >= L2 (equality only on axis-aligned paths)")

## Part 2: Unit Circles Under Different Norms

A powerful way to understand norms is to visualize their **unit circles**—the set of all points at distance 1 from the origin.

- L2 unit circle: The familiar round circle
- L1 unit circle: A diamond (rotated square)

*"The Pickbox Man's world is made of diamonds. Every step must be along a tunnel—no cutting corners."*

In [ ]:
# Visualize unit circles for different norms
fig, ax = plt.subplots(figsize=(10, 10))

theta = np.linspace(0, 2*np.pi, 100)

# L2 unit circle (standard circle)
x_l2 = np.cos(theta)
y_l2 = np.sin(theta)
ax.plot(x_l2, y_l2, 'g-', linewidth=3, label='L2 unit circle (Euclidean)')

# L1 unit circle (diamond)
x_l1 = np.array([1, 0, -1, 0, 1])
y_l1 = np.array([0, 1, 0, -1, 0])
ax.plot(x_l1, y_l1, 'purple', linewidth=3, label='L1 unit circle (Manhattan)')

# L-infinity unit circle (square)
x_linf = np.array([1, 1, -1, -1, 1])
y_linf = np.array([1, -1, -1, 1, 1])
ax.plot(x_linf, y_linf, 'orange', linewidth=3, linestyle='--', label='L∞ unit circle (max norm)')

ax.scatter(0, 0, s=100, c='black', zorder=5)
ax.set_xlim(-1.5, 1.5)
ax.set_ylim(-1.5, 1.5)
ax.set_xlabel('x', fontsize=12)
ax.set_ylabel('y', fontsize=12)
ax.set_title('Unit Circles Under Different Norms\n(All points at distance 1 from origin)', fontsize=13)
ax.legend(fontsize=11, loc='upper right')
ax.set_aspect('equal')
ax.axhline(0, color='black', linewidth=0.5)
ax.axvline(0, color='black', linewidth=0.5)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Key insight: The 'shape' of distance changes with the norm!")
print("  - L2: Distance is the same in all directions")
print("  - L1: Diagonal directions are 'further' than axis-aligned")
print("  - L∞: Only the largest coordinate matters")

## Part 3: Creature Distances Under Different Norms

Let's see how the choice of norm affects creature similarity rankings. Which creatures are "closest" to the Witch Creature depends on how we measure distance!

*"The Colonel asks which creatures are most like the Witch Creature, so we know what else lurks in its territory. But 'most like' depends on how you measure."*  
— Expedition planning notes

In [ ]:
# Extract behavioral features
behavioral_features = ['aggression', 'sociality', 'nocturnality', 'territoriality', 'hunting_strategy']
X = creature_vectors[behavioral_features].values
names = creature_vectors['common_name'].values

# Find Witch Creature index
witch_idx = np.where(names == 'Witch Creature')[0][0]
witch_vector = X[witch_idx].reshape(1, -1)

# Calculate distances using different norms
l1_distances = cdist(witch_vector, X, metric='cityblock')[0]
l2_distances = cdist(witch_vector, X, metric='euclidean')[0]
linf_distances = cdist(witch_vector, X, metric='chebyshev')[0]  # L-infinity

# Create comparison DataFrame
comparison = pd.DataFrame({
    'creature': names,
    'L1_dist': l1_distances,
    'L2_dist': l2_distances,
    'Linf_dist': linf_distances
})

# Rank by each distance
comparison['L1_rank'] = comparison['L1_dist'].rank().astype(int)
comparison['L2_rank'] = comparison['L2_dist'].rank().astype(int)
comparison['Linf_rank'] = comparison['Linf_dist'].rank().astype(int)

# Exclude Witch Creature itself (distance 0)
comparison = comparison[comparison['creature'] != 'Witch Creature']

print("Distance to Witch Creature — Different Norms Give Different Rankings!")
print("="*85)
print(comparison.sort_values('L2_dist')[['creature', 'L1_dist', 'L2_dist', 'Linf_dist', 
                                          'L1_rank', 'L2_rank', 'Linf_rank']].to_string(index=False))

In [ ]:
# Highlight ranking differences
print("\nTop 5 Most Similar to Witch Creature by Each Norm:")
print("="*60)

print("\nL1 (Manhattan):")
for _, row in comparison.nsmallest(5, 'L1_dist').iterrows():
    print(f"  {row['creature']:25} distance = {row['L1_dist']:.3f}")

print("\nL2 (Euclidean):")
for _, row in comparison.nsmallest(5, 'L2_dist').iterrows():
    print(f"  {row['creature']:25} distance = {row['L2_dist']:.3f}")

print("\nL∞ (Max):")
for _, row in comparison.nsmallest(5, 'Linf_dist').iterrows():
    print(f"  {row['creature']:25} distance = {row['Linf_dist']:.3f}")

### Why Do Rankings Differ?

Different norms weight the feature differences differently:

- **L2** penalizes large differences more heavily (squared differences)
- **L1** treats all differences linearly (no extra penalty for big gaps)
- **L∞** only cares about the single largest difference

This matters! If one creature differs on a single trait dramatically, L∞ will flag it as distant, while L1 might not.

In [ ]:
# Show feature-level breakdown for selected creatures
print("Feature-Level Comparison: Witch Creature vs Selected Creatures")
print("="*80)

witch_features = X[witch_idx]
print(f"\nWitch Creature: {witch_features}")
print(f"Features: {behavioral_features}")

compare_creatures = ['Maw Beast', 'Stakdur', 'Cave Bat', 'Yeller Frog']

for creature_name in compare_creatures:
    idx = np.where(names == creature_name)[0][0]
    features = X[idx]
    diffs = np.abs(witch_features - features)
    
    print(f"\n{creature_name}:")
    print(f"  Features:    {features}")
    print(f"  |Diff|:      {diffs.round(2)}")
    print(f"  L1 = sum(|diff|) = {diffs.sum():.3f}")
    print(f"  L2 = sqrt(sum(diff²)) = {np.sqrt(np.sum(diffs**2)):.3f}")
    print(f"  L∞ = max(|diff|) = {diffs.max():.3f}")

## Part 4: Norm Sensitivity to Outliers

A crucial difference: **L2 is more sensitive to large differences** than L1.

This has practical implications:
- L2 heavily penalizes outliers (one big difference dominates)
- L1 treats all differences equally (more robust to outliers)

*"When measuring expedition success, do you penalize one disastrous outcome more than five small failures? The Colonel would say yes—one catastrophe outweighs minor setbacks."*

In [ ]:
# Demonstrate outlier sensitivity
print("Outlier Sensitivity: L1 vs L2")
print("="*60)

# Case 1: Many small differences
vec_a = np.array([0.0, 0.0, 0.0, 0.0, 0.0])
vec_b = np.array([0.2, 0.2, 0.2, 0.2, 0.2])  # All features differ by 0.2

l1_case1 = np.sum(np.abs(vec_b - vec_a))
l2_case1 = np.sqrt(np.sum((vec_b - vec_a)**2))

print("\nCase 1: Many small differences")
print(f"  Vector A: {vec_a}")
print(f"  Vector B: {vec_b}")
print(f"  L1 distance: {l1_case1:.4f}")
print(f"  L2 distance: {l2_case1:.4f}")

# Case 2: One large difference (same L1 sum)
vec_c = np.array([1.0, 0.0, 0.0, 0.0, 0.0])  # One feature differs by 1.0

l1_case2 = np.sum(np.abs(vec_c - vec_a))
l2_case2 = np.sqrt(np.sum((vec_c - vec_a)**2))

print("\nCase 2: One large difference (same L1 total)")
print(f"  Vector A: {vec_a}")
print(f"  Vector C: {vec_c}")
print(f"  L1 distance: {l1_case2:.4f}")
print(f"  L2 distance: {l2_case2:.4f}")

print("\n" + "="*60)
print("Key insight: L1 distances are equal, but L2 treats them differently!")
print(f"  L2 ratio (Case2/Case1): {l2_case2/l2_case1:.2f}x")
print("  L2 penalizes the concentrated outlier more heavily.")

## Part 5: Visualizing Distance Contours

Let's visualize what "nearby" means under different norms by plotting distance contours from a reference creature.

In [ ]:
# Create 2D visualization: aggression vs territoriality
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Reference point: Witch Creature
ref_x, ref_y = creature_vectors[creature_vectors['common_name'] == 'Witch Creature'][['aggression', 'territoriality']].values[0]

# Create grid
xx, yy = np.meshgrid(np.linspace(0, 1, 100), np.linspace(0, 1, 100))

# L2 distances
l2_grid = np.sqrt((xx - ref_x)**2 + (yy - ref_y)**2)

# L1 distances
l1_grid = np.abs(xx - ref_x) + np.abs(yy - ref_y)

# Plot L2 contours
ax = axes[0]
contour_l2 = ax.contour(xx, yy, l2_grid, levels=[0.2, 0.4, 0.6, 0.8], colors='green', linewidths=2)
ax.clabel(contour_l2, inline=True, fontsize=10, fmt='%.1f')

# Plot creatures
for _, row in creature_vectors.iterrows():
    color = 'red' if row['common_name'] == 'Witch Creature' else 'steelblue'
    size = 200 if row['common_name'] == 'Witch Creature' else 80
    ax.scatter(row['aggression'], row['territoriality'], c=color, s=size, edgecolor='black', alpha=0.7)
    if row['common_name'] in ['Witch Creature', 'Maw Beast', 'Stakdur', 'Cave Bat']:
        ax.annotate(row['common_name'], (row['aggression']+0.02, row['territoriality']+0.02), fontsize=9)

ax.set_xlabel('Aggression', fontsize=12)
ax.set_ylabel('Territoriality', fontsize=12)
ax.set_title('L2 (Euclidean) Distance Contours from Witch Creature', fontsize=12)
ax.set_xlim(-0.05, 1.05)
ax.set_ylim(-0.05, 1.05)

# Plot L1 contours
ax = axes[1]
contour_l1 = ax.contour(xx, yy, l1_grid, levels=[0.2, 0.4, 0.6, 0.8], colors='purple', linewidths=2)
ax.clabel(contour_l1, inline=True, fontsize=10, fmt='%.1f')

# Plot creatures
for _, row in creature_vectors.iterrows():
    color = 'red' if row['common_name'] == 'Witch Creature' else 'steelblue'
    size = 200 if row['common_name'] == 'Witch Creature' else 80
    ax.scatter(row['aggression'], row['territoriality'], c=color, s=size, edgecolor='black', alpha=0.7)
    if row['common_name'] in ['Witch Creature', 'Maw Beast', 'Stakdur', 'Cave Bat']:
        ax.annotate(row['common_name'], (row['aggression']+0.02, row['territoriality']+0.02), fontsize=9)

ax.set_xlabel('Aggression', fontsize=12)
ax.set_ylabel('Territoriality', fontsize=12)
ax.set_title('L1 (Manhattan) Distance Contours from Witch Creature', fontsize=12)
ax.set_xlim(-0.05, 1.05)
ax.set_ylim(-0.05, 1.05)

plt.tight_layout()
plt.show()

print("Notice: L2 contours are circles; L1 contours are diamonds.")
print("Creatures in different positions relative to contours under each norm!")

## Part 6: Norms in Machine Learning — Regularization

The L1 and L2 norms appear throughout machine learning, especially in **regularization**:

| Regularization | Norm | Effect | ML Name |
|---------------|------|--------|----------|
| L2 | $\|\mathbf{w}\|_2^2 = \sum w_i^2$ | Shrinks all weights toward zero | **Ridge** |
| L1 | $\|\mathbf{w}\|_1 = \sum |w_i|$ | Drives some weights exactly to zero | **Lasso** |

The geometric reason: L1's diamond shape has corners on the axes, making it more likely to "hit" at a point where some coordinates are exactly zero.

*"The Archives found that when classifying manuscripts, some features matter immensely while others are noise. L1 regularization automatically identifies which features to ignore—it's like the algorithm learns to focus on what's important."*  
— Computational stylometry notes

In [ ]:
# Demonstrate why L1 leads to sparsity
fig, ax = plt.subplots(figsize=(10, 10))

# Draw L1 unit ball (diamond)
x_l1 = np.array([1, 0, -1, 0, 1])
y_l1 = np.array([0, 1, 0, -1, 0])
ax.fill(x_l1, y_l1, alpha=0.3, color='purple', label='L1 constraint region')
ax.plot(x_l1, y_l1, 'purple', linewidth=2)

# Draw L2 unit ball (circle)
theta = np.linspace(0, 2*np.pi, 100)
ax.fill(np.cos(theta), np.sin(theta), alpha=0.3, color='green', label='L2 constraint region')
ax.plot(np.cos(theta), np.sin(theta), 'green', linewidth=2)

# Draw some objective function contours (ellipses centered off-origin)
center = np.array([0.8, 0.6])
for r in [0.3, 0.5, 0.7, 0.9, 1.1]:
    ellipse_x = center[0] + r * np.cos(theta)
    ellipse_y = center[1] + r * np.sin(theta)
    ax.plot(ellipse_x, ellipse_y, 'gray', linewidth=1, alpha=0.5)

ax.scatter(*center, s=100, c='red', marker='*', zorder=10, label='Unconstrained optimum')

# Mark where constraints meet objective
ax.scatter(1, 0, s=150, c='purple', marker='o', zorder=10, edgecolor='black', linewidth=2)
ax.annotate('L1 solution\n(sparse: w₂=0)', (1.05, 0.05), fontsize=11, fontweight='bold')

ax.scatter(0.8, 0.6, s=150, c='green', marker='o', zorder=10, edgecolor='black', linewidth=2)
ax.annotate('L2 solution\n(both weights ≠ 0)', (0.65, 0.75), fontsize=11, fontweight='bold')

ax.set_xlim(-1.5, 1.8)
ax.set_ylim(-1.5, 1.5)
ax.set_xlabel('Weight w₁', fontsize=12)
ax.set_ylabel('Weight w₂', fontsize=12)
ax.set_title('Why L1 Regularization Creates Sparse Solutions\n(Constraint touches objective at corner → w₂=0)', fontsize=13)
ax.legend(loc='lower left', fontsize=10)
ax.set_aspect('equal')
ax.axhline(0, color='black', linewidth=0.5)
ax.axvline(0, color='black', linewidth=0.5)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Part 7: When to Use Which Norm

| Situation | Recommended Norm | Reason |
|-----------|-----------------|--------|
| Standard distance/similarity | L2 | Intuitive "straight line" interpretation |
| Robust to outliers | L1 | Less sensitive to extreme values |
| Feature selection needed | L1 (Lasso) | Drives irrelevant weights to zero |
| All features likely important | L2 (Ridge) | Shrinks all weights, keeps all |
| High-dimensional data | L1 or L2 | L1 for sparsity, L2 for stability |
| Grid/graph distances | L1 | Natural for networks, city blocks |

In [ ]:
# Practical example: Which norm for creature classification?
print("Practical Decision: Classifying Dangerous vs Safe Creatures")
print("="*65)

# Define dangerous creatures
dangerous = ['Witch Creature', 'Stakdur', 'Maw Beast', 'Wharver', 'Marsh Hornet']
creature_vectors['is_dangerous'] = creature_vectors['common_name'].isin(dangerous)

# Test creature: Stone Spine Lizard
test_creature = 'Stone Spine Lizard'
test_idx = creature_vectors[creature_vectors['common_name'] == test_creature].index[0]
test_vector = X[test_idx].reshape(1, -1)

# Find k=3 nearest neighbors under each norm
l1_dists = cdist(test_vector, X, metric='cityblock')[0]
l2_dists = cdist(test_vector, X, metric='euclidean')[0]

# Exclude self
l1_dists[test_idx] = np.inf
l2_dists[test_idx] = np.inf

k = 3
l1_neighbors = np.argsort(l1_dists)[:k]
l2_neighbors = np.argsort(l2_dists)[:k]

print(f"\nClassifying: {test_creature}")
print(f"Actual status: {'Dangerous' if creature_vectors.loc[test_idx, 'is_dangerous'] else 'Safe'}")

print(f"\n3-NN using L1 (Manhattan):")
l1_dangerous_count = 0
for idx in l1_neighbors:
    name = names[idx]
    status = 'DANGEROUS' if creature_vectors.loc[idx, 'is_dangerous'] else 'safe'
    if status == 'DANGEROUS':
        l1_dangerous_count += 1
    print(f"  {name:25} - {status}")
print(f"  Prediction: {'DANGEROUS' if l1_dangerous_count >= 2 else 'safe'} (majority vote)")

print(f"\n3-NN using L2 (Euclidean):")
l2_dangerous_count = 0
for idx in l2_neighbors:
    name = names[idx]
    status = 'DANGEROUS' if creature_vectors.loc[idx, 'is_dangerous'] else 'safe'
    if status == 'DANGEROUS':
        l2_dangerous_count += 1
    print(f"  {name:25} - {status}")
print(f"  Prediction: {'DANGEROUS' if l2_dangerous_count >= 2 else 'safe'} (majority vote)")

## Summary

| Concept | Key Insight | Densworld Example |
|---------|-------------|-------------------|
| **L2 (Euclidean)** | Straight-line distance; penalizes large differences | Vagabu Olt measuring "as the crow flies" |
| **L1 (Manhattan)** | Grid distance; robust to outliers | The Pickbox Man navigating tunnels |
| **Unit Circles** | L2 = circle, L1 = diamond | Shape of "nearby" changes with norm |
| **Ranking Changes** | Different norms → different nearest neighbors | Witch Creature's neighbors depend on norm |
| **Outlier Sensitivity** | L2 more sensitive; L1 more robust | One extreme trait vs. many mild ones |
| **Regularization** | L1 → sparsity (Lasso); L2 → shrinkage (Ridge) | Feature selection for manuscript analysis |

---

## Exercises

### Exercise 1: Rank Comparison

Choose a creature (not Witch Creature) and find its 5 nearest neighbors under both L1 and L2 norms. Do the rankings differ? Which creatures change rank the most?

In [ ]:
# Exercise 1: Your code here
# Hint: Follow the pattern from Part 3



### Exercise 2: Create a Scenario

Create two synthetic 5-dimensional vectors where:
- L1 distance is 2.0
- L2 distance is very different (either much larger or much smaller than L1)

Explain why this happens.

In [ ]:
# Exercise 2: Your code here
# Hint: Think about concentrated vs. spread-out differences



### Exercise 3: Boundary Observations

Using the `dens_boundary` dataset, calculate the L1 and L2 norms of the measurement error vectors (treat each observation as a point in feature space using `stability_index` and `measurement_error`). Which observations have the largest norms? Are they the same under both norms?

In [ ]:
# Exercise 3: Your code here
# Hint: np.abs() for L1, np.sqrt() for L2



### Exercise 4: K-NN Classification

Implement a simple k-NN classifier that can switch between L1 and L2 distance. Test it on classifying creatures as "high_aggression" (aggression > 0.5) using all behavioral features. Does the choice of norm affect accuracy?

In [ ]:
# Exercise 4: Your code here
# Hint: Use leave-one-out cross-validation



---

## Next Lesson

In **Lesson 3: The Dot Product and Similarity**, we'll explore a different way to measure how "alike" two vectors are—not by their distance, but by their **alignment**. The dot product reveals whether creatures (or manuscripts) "point in the same direction" in feature space.

*"Distance tells you how far apart things are. The dot product tells you whether they're pointed the same way. A Marsh Hornet and a Stakdur may be distant in space, but they share the same predatory intent."*  
— Boffa Trent